# SSRS

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
#plt.rcParams["figure.figsize"] = (width,height)
#sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from d2e2f.visualization.visualize import plot_map, plot_trips

In [ ]:
%reload_kedro
ship = 'SSRS15-01_322012079' # west coast
#ship = 'SSRS15-04_322012068' #(bad)
#ship = 'SSRS15-03_322012059' # east coast

data_selected = catalog.load("ssrs.data_selected")[ship]()

datas_moving = catalog.load("ssrs.data_moving")
data_moving = datas_moving[ship]()

datas_running = catalog.load("ssrs.data_running")
data_running = datas_running[ship]()

In [ ]:
signal = catalog.load("ssrs.telematikenheter")[f"{ship}/latitude"]()
signal.index = pd.to_datetime(signal.index)
signal.sort_index(inplace=True)

In [ ]:
signal['time'] = (signal.index - signal.index[0]).total_seconds()
signal['time'].diff().hist(bins=100)

In [ ]:
raw_data = catalog.load("ssrs.data")[ship]()

In [ ]:
raw_data.head()

In [ ]:
raw_data.isnull().any()

In [ ]:
raw_data['time'] = (raw_data.index - raw_data.index[0]).total_seconds()
assert not (raw_data['time'].diff() < 0 ).any()

mask = raw_data['time'].diff() > 1
jumps = raw_data.loc[mask]
i = raw_data.index.get_loc(jumps.index[0])

In [ ]:
raw_data.index[i-1]

In [ ]:
raw_data.index[i]

In [ ]:
(raw_data.index[i] - raw_data.index[i-1]).total_seconds()

In [ ]:
df_ = pd.concat((raw_data.iloc[0:3],raw_data.iloc[100:103], raw_data.iloc[300:303]))
#mask = df_['time'].diff() > 1
#df_['time'] = 
#mask = (df_.index - df_.index[0]).total_seconds() > 1
mask = np.concatenate(([True],(df_.index[1:] - df_.index[0:-1]) > "1S"))
n_groups = mask.sum()
df_.loc[mask,'group'] = np.arange(0,n_groups) 
df_['group'] = df_['group'].ffill()
#df_['group'].fillna(0, inplace=True)
groups = []
for group, df_group in df_.groupby(by=['group'], sort=False):
    groups.append(df_group.resample('0.5S').interpolate())
    
df_resample = pd.concat(groups)

In [ ]:
df_resample[['time','group']]

In [ ]:
%reload_kedro
statistics = catalog.load("ssrs.statistics")[ship]()
statistics_moving = catalog.load("ssrs.statistics_moving")[ship]()
statistics_running = catalog.load("ssrs.statistics_running")[ship]()
model = catalog.load("ssrs.model_running")[ship]()
models = catalog.load("ssrs.model_running")


In [ ]:
plot_map(df=data_moving, zoom_start=9, color_key='sog', colormap=['black','red'])

In [ ]:
data_running['sog'].mean()

In [ ]:
sns.pairplot(data_moving.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), 
             kind='scatter', plot_kws={'alpha':0.1}, y_vars=['fuel_rate'])


In [ ]:
sns.pairplot(data_running.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), 
             kind='scatter', plot_kws={'alpha':0.1}, y_vars=['fuel_rate'])

In [ ]:
statistics

In [ ]:
statistics_moving

In [ ]:
statistics_running

In [ ]:
data = data_running.copy()
data['sog_kt'] = data['sog']*3.6/1.852
X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data['sog'].min(), data['sog'].max(), 100)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']*1000/3600*1852

In [ ]:
fig,axes=plt.subplots(nrows=2)
ax =axes[0]
data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
data.resample('10S').mean().plot(x='sog', y='fuel_per_m', style='.', alpha=0.1, ax=ax, label='__no_label__')

ax.set_ylim(data['fuel_per_m'].quantile(0.01), 
            data['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='data')

prediction.plot(x='sog', y='fuel_per_m', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data['sog'].max())),1));
ax.grid(True)
ax.legend()
ax.set_ylabel('Consumption per nautical mile [L/NM]')
ax.set_xlabel('Speed over ground [m/s]')


ax = axes[1]
data['sog_kt'].hist(bins=50, density=True, ax=ax)
ax.set_xticks(np.arange(0,int(np.ceil(data['sog_kt'].max())),2));
ax.set_xlabel('sog [m/s]');
ax.set_ylabel('Probability density')
ax.set_xlabel('Speed over ground [kts]')

In [ ]:
fig,ax=plt.subplots()
data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
data.resample('10S').mean().plot(x='sog_kt', y='fuel_per_m', style='.', alpha=0.10, ax=ax, label='__no_label__')

ax.set_ylim(data['fuel_per_m'].quantile(0.01), 
            data['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='data')

for key, loader in models.items():

    model_ = loader()
    prediction_ = X_pred.copy()
    prediction_['fuel_rate'] = model_.predict(X_pred)
    prediction_['fuel_per_m'] = prediction_['fuel_rate']/X_pred['sog']*1000/3600*1852
    prediction_['sog_kt'] = prediction_['sog']*3.6/1.852
    prediction_.plot(x='sog_kt', y='fuel_per_m', ax=ax, label=key)



ax.set_xticks(np.arange(0,int(np.ceil(data['sog_kt'].max())),2));
ax.grid(True)
ax.legend()
ax.set_ylabel('Consumption per nautical mile [L/NM]')
ax.set_xlabel('Speed over ground [kts]')

In [ ]:
data_selected['sog_kt'] = data_selected['sog']*3.6/1.852
data_selected['fuel_per_m'] = data_selected['fuel_rate']/data_selected['sog']*1000/3600*1852
data_selected['sog_kt_round'] = 2*(1/2*data_selected['sog_kt']).round(decimals=0)
df_speed_table = data_selected.groupby(by='sog_kt_round').mean()
df_speed_table_std = data_selected.groupby(by='sog_kt_round').std()


In [ ]:
df_speed_table.plot(x='sog_kt', y='fuel_per_m')

In [ ]:
fig,ax=plt.subplots()
df_speed_table.plot(x='sog_kt', y='fuel_per_m', style='k-', ax=ax)
y1 = (df_speed_table - df_speed_table_std)['fuel_per_m']
y2 = (df_speed_table + df_speed_table_std)['fuel_per_m']


ax.fill_between(df_speed_table['sog_kt'], y1, y2, zorder=-10, color='lightgray', label='+/-std')
ax.set_ylim(0,15.0)
ax.legend()
ax.grid(True)

In [ ]:
speed_table = df_speed_table[['sog_kt','rpm','fuel_rate','fuel_per_m']].copy()
speed_table['sog_kt'] = speed_table['sog_kt'].round(decimals=1)
speed_table['rpm'] = speed_table['rpm'].round(decimals=0)
speed_table['fuel_rate'] = (1000*speed_table['fuel_rate']).round(decimals=0)
speed_table['fuel_per_m'] = (speed_table['fuel_per_m']).round(decimals=0)
speed_table